In [69]:
import sqlite3
import pandas as pd
import os, sys, re, string
import itertools

In [2]:
"""
This module can be used to add from a parent folder all the excel files to a sqlite3 database.
The class requires the database name and the path where you have the excel files.

Warning: If it founds an .csv file it will save to database only the first sheet!
Requires pandas module to be installed.

"""

'\nThis module can be used to add from a parent folder all the excel files to a sqlite3 database.\nThe class requires the database name and the path where you have the excel files.\n\nWarning: If it founds an .csv file it will save to database only the first sheet!\nRequires pandas module to be installed.\n\n'

In [205]:
class Df2db:
    
    def __init__(self, dbname, root_path):
        self.dbname = dbname
        self.root_path = root_path    
    
    def connect_db(self):
        #Connect to a db and if it not exists creates one with the name given
        connection = sqlite3.connect(self.dbname)
        cursor = connection.cursor()
        return connection, cursor
    
    def close(self):
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        connection.commit()
        connection.close()
    
    def norm_pctmarks(self, astring):
        #Removing punctuation marks from the string, necessary for making compatible table names
        punctuation_marks = list(str(string.punctuation).replace('_', ''))+[' ']
        try:
            for char in punctuation_marks:
                astring = astring.replace(char, '_')
        except:
            pass
        return astring
    
    def rename_duplicate_dfcols(self, df):
        #Rename DF columns if found duplicates (credit to SO"Lamakaha")
        try:
            cols=pd.Series(df.columns)
            for dup in df.columns.get_duplicates(): 
                cols[df.columns.get_loc(dup)]=[dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(df.columns.get_loc(dup).sum())]
                df.columns=cols
        except Exception as e:
            print("Got ", e)
            pass
        return df
    
    
    def save_tosql(self,connection, df_sht, sht, path_to_xlname):
        
        xlname = path_to_xlname.strip().split('\\')[-1]
        tablename_insql = str(xlname + '_ON_' + sht)
        
        #Remove punctuation marks from table name
        tablename_insql = Df2db(self.dbname, self.root_path).norm_pctmarks(tablename_insql)
        
        #Replacing spaces with underscores to be compatible to sql tables
        df_sht.rename(columns=lambda x: x.strip().replace(' ', '_'), inplace=True)
        
        #Rename duplicate columns from df
        df_sht = Df2db(self.dbname, self.root_path).rename_duplicate_dfcols(df_sht)
        
        #Saving df to sqlite3 db
        df_sht.to_sql(tablename_insql, connection, if_exists="replace", index=False)
        connection.commit()
        print('{} ok'.format(tablename_insql))
        
        
    
    
    def xltosql(self, path_to_xlname):
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        df = pd.ExcelFile(path_to_xlname)
        for sht in df.sheet_names:
            df_sht = df.parse(sht)
            if df_sht.shape == (0,0): 
                pass
            else:
                print("has")
                Df2db(self.dbname, self.root_path).save_tosql(connection, df_sht,sht,path_to_xlname)
                           
    def dftosql(self, df, sht='Sheet1'):
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        Df2db(self.dbname, self.root_path).save_tosql(connection, df_sht,sht,path_to_xlname)
        
    def csvtosql(self, path_to_xlname):
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        df
        Df2db(self.dbname, self.root_path).save_tosql(connection, df, sht, path_to_xlname)
        
    
    def df_tosql(self, path_to_xlname, dfname=''):
        #Get thru all sheets and if it has data save it to db
        filename = path_to_xlname.split('\\')[-1]
    
        if str(type(path_to_xlname)) == "<class 'pandas.core.frame.DataFrame'>":
            print('df')
            Df2db(self.dbname, self.root_path).dftosql(path_to_xlname)
        
        elif re.search('.xls', filename):
            print('.xls')
            Df2db(self.dbname, self.root_path).xltosql(path_to_xlname)
                
            
        elif re.search('.csv', filename):
            print('.csv')
    
    
            
    def getdf_fromdb(self, table_name):
        #gets the table from the db
        #the table name must have this format excel_xlname_sheet_sheetname
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        query = "SELECT * FROM {}".format(table_name)
        df = pd.read_sql_query(query, connection)
        return df

    def show_db_tables(self):
        #Shows all tables names from the db
        connection, cursor = Df2db(self.dbname, self.root_path).connect_db()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        all_tb = cursor.fetchall()
        #print(all_tb)
        all_tb = [x[0] for x in all_tb]
        column_name = '{}_TABLES'.format(self.dbname)
        df = pd.DataFrame({column_name : all_tb})
        try:
            df.to_csv(column_name+'.csv', index=False)
        except Exception as e:
            input('CSV already openned please close it..\n {}'.format(e))
            sys.exit()
        print('{} tables in {}\n'.format(len(all_tb), self.dbname))
        return all_tb
        
        
        
    def drop_tablefrom_db(self, table_name):
        #Using the cursor created in connect_db func executes statement
        connection = sqlite3.connect(self.dbname)
        cursor = connection.cursor()
        query = 'DROP TABLE {};'.format(table_name)
        try:
            cursor.execute(query)
        except Exception as e:
            print("Table does not exist!\nPress Enter to exit...")
            input(e)
            sys.exit()
    
    
    def getfilespath_from(self):
        #Walk thru a start path and return a list of paths to files
        allfiles = []
        for root, dirs, files in os.walk(self.root_path):
            for file in files:
                path_tofile = root + '\\' + file
                allfiles.append(path_tofile)
        return allfiles

    
    def get_dfpaths(self):
        #get a list of paths to excel files (.xlsx, .xls, .xlsm, .csv)
        paths_tofiles = Df2db(self.dbname, self.root_path).getfilespath_from()
        filtered_extensions = []
        for path in paths_tofiles:
            p = path.split('\\')
            if re.search('.xls', p[-1]) or re.search('.csv', p[-1]):
                filtered_extensions.append(path)
        return filtered_extensions
    
    def dfs_tosql(self):
        #Run thru alldfs and save them to db
        path_to_dfs = Df2db(self.dbname, self.root_path).get_dfpaths()
        print('\nThere are ',len(path_to_dfs), ' files to be added..\n\n')
        for df in path_to_dfs:
            #print(df)
            Df2db(self.dbname, self.root_path).df_tosql(df)
        #tab = Df2db(self.dbname, self.root_path).show_db_tables()
        
        

In [206]:
todf = Df2db('dxl.db', r'D:\working_python\pandas-dataframe-to-sqlite3-database-master\xlsm\ex')
todf.dfs_tosql()


There are  3  files to be added..


.xls
has
5_8_Copy_a_Worksheet_to_a_new_Workbook_xlsm_ON_Sheet1 ok
.csv
.csv


In [ ]:
todf.df_tosql()

In [140]:
st = r"D:\working_python\pandas-dataframe-to-sqlite3-database-master\xlsm\ex\5.8 Copy a Worksheet to a new Workbook.xlsm"
st.split('\\')[-1]

'5.8 Copy a Worksheet to a new Workbook.xlsm'

In [101]:
df = pd.read_excel(r"D:\working_python\pandas-dataframe-to-sqlite3-database-master\xlsm\ex\5.8 Copy a Worksheet to a new Workbook.xlsm")
df.head()

,Macro 23: Copy a Worksheet to a new Workbook,Unnamed: 1,Unnamed: 2
0,NaN,Creates a new workbook and copies this sheet t...,NaN
1,NaN,Press ALT+F11 to see the code,NaN
2,NaN,January,26262.739215
3,NaN,February,28911.674142
4,NaN,March,52149.123907


In [203]:
#Make a dataframe
import numpy as np
import pandas as pd
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

#Save it to sql, you must give the df and a name for the df
todf.df_tosql(df, 'test')

AttributeError: 'DataFrame' object has no attribute 'split'

In [204]:
todf.show_db_tables()

1 tables in dxl.db



['5_8_Copy_a_Worksheet_to_a_new_Workbook_xlsm_SHEET_Sheet1']

In [37]:
todf.getdf_fromdb("EXCEL_6_7_Deleting_Blank_Columns_xlsm_SHEET_Sheet1")

,Macro_38:_Deleting_Blank_Columns,Unnamed:_1,Unnamed:_2,Unnamed:_3
0,None,Press Alt+F11 to Review the Code,None,NaN
1,None,None,None,NaN
2,None,January,None,3224.179155
3,None,February,None,2758.838342
4,None,March,None,1301.704898
5,None,April,None,2316.694080
6,None,May,None,2677.077357
7,None,June,None,3015.299971
8,None,July,None,8892.723208
9,None,August,None,185.531620


In [38]:
con, c = todf.connect_db()
con.close()

In [ ]:
"""
#Import class Df2db from df2db module
from df2db import Df2db

#Instantiate with the database name and the path where you got excel files
todf = Df2db('dbname.db', r'D:\alot_of_xlfiles')

#Call dfs_tosql function to search in the path you give for excel files 
#and save them to the database with the name given
todf.dfs_tosql()

#Show tables from the database
todf.show_db_tables()

#Get a table from the database as a dataframe object
#in order to use it in pandas for manipulation
todf.getdf_fromdb("EXCEL_xl4_xlsx_SHEET_Sheet1")

#Delete a table from the database
todf.drop_tablefrom_db("EXCEL_xl4_xlsx_SHEET_Sheet1")

#Show tables from db, now you see one it's gone
todf.show_db_tables()

#Close the connection when you are done.
todf.close()

#Instantiate with the database name and the path where you got excel files
todf = Df2db('dbname.db', r'D:\alot_of_xlfiles')

#Make a dataframe
import numpy as np
import pandas as pd
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

#Save it to sql, you must give the df and a name for the df
todf.df_tosql(df, 'test')

#Show current tables from db 
todf.show_db_tables()
#Close when you are done
todf.close()

#Each table saved in database will have this form
tablname = "EXCEL_test_xlsx_SHEET_Sheet1"
tablname.split('_')


'EXCEL' - all tables in db will start with this prefix
'test' - the name if the excel or df 
'xlsx' - the extension of the file
'SHEET' - the next folowing this will be the Sheet name of the excel
That's because it looks in the workbook in all sheets for tables and saves the sheet name also
'Sheet1' - The sheet where the table was found

If the excel or df has punctuation marks or spaces those will be replaced with underscore "_"
This replace is done in order to be compatible with sqlite3 database
Also, if the column names contains spaces those will be replaced with "_"

"""